In [1]:
import numpy as np
import math

# Metallicity Grid

In [9]:
Z_SOL = 0.0142 # COMPAS Default

zminn = 0.000100 # COMPAS MIN
zmaxx = 0.030000 # COMPAS MAX

Z_MIN_LOG = np.log10(zminn)
Z_MAX_LOG = np.log10(zmaxx)

metallicities = np.logspace(Z_MIN_LOG, Z_MAX_LOG, 10)
print(metallicities)

[0.0001     0.00018847 0.0003552  0.00066943 0.00126166 0.00237782
 0.0044814  0.00844598 0.01591789 0.03      ]


# Star Forming Mass Grid

In [10]:
SF_MIN_LOG = 3 # 1e3 M_sol
SF_MAX_LOG = 9 # 1e9 M_sol

sf_mass = np.logspace(SF_MIN_LOG, SF_MAX_LOG, 7)
print(sf_mass)

n_tot_binaries = sf_mass / 20.0 # Got this by looking at the average mass in a stellar binary drawn using Kroupa
print(n_tot_binaries)

[1.e+03 1.e+04 1.e+05 1.e+06 1.e+07 1.e+08 1.e+09]
[5.e+01 5.e+02 5.e+03 5.e+04 5.e+05 5.e+06 5.e+07]


# Writing the Sbatch Files

In [11]:
def getSbatch(n_systems, metallicities=[0.0001], run_size=10000, job_name='cosmo_bh', queue='development', timeout = '02:00:00'):  
    n_systems = int(n_systems)
    run_size = int(run_size)
    
    n_runs = int(n_systems/run_size)
    
    tasks = n_runs*len(metallicities)
    nodes = max(1, math.ceil(tasks/56))
    
        
    preamble_text = f"""#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

mkdir $WORK/cosmo_bh_grid/n_{n_systems:.2e}

# Launch MPI code...
"""
    for met in metallicities:
        preamble_text+= f'mkdir $WORK/cosmo_bh_grid/n_{n_systems:.2e}/met_{met:.2e}\n' 

    preamble_text += f"\nfor i in `seq 1 {n_runs}` \ndo \n"
    
    for met in metallicities:
        output_path = f'$WORK/cosmo_bh_grid/n_{n_systems:.2e}/met_{met:.2e}'
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --metallicity {met:.8f}"
        preamble_text += f"    ibrun -n 1 ~/COMPAS/src/COMPAS" + specs + f" --output-container run_$i & \n"
    
    preamble_text += "done \nwait"
    
    return preamble_text



In [12]:
metallicities[0]

0.0001

In [5]:
n_binaries = n_tot_binaries[5]

met_min = 9 # MIN = 1
met_max = 10 # MAX = 10

job_name = f"n_{n_binaries:.2e}_met_{metallicities[met_min]:.2e}_to_{metallicities[met_max-1]:.2e}"
print(f"Job: {job_name}\n")

sbatch_text = getSbatch(n_binaries, metallicities[met_min:met_max], run_size=500000, job_name=job_name, queue='small', timeout='48:00:00')
print(sbatch_text)

# file1 = open(f"sbatch_cosmo/{job_name}", "w")  # write mode
# file1.write(sbatch_text)
# file1.close()

Job: n_5.00e+06_met_3.00e-02_to_3.00e-02

#!/bin/bash
## COMPAS Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J n_5.00e+06_met_3.00e-02_to_3.00e-02           # Job name
#SBATCH -o n_5.00e+06_met_3.00e-02_to_3.00e-02.o%j	   # Name of stdout output file
#SBATCH -e n_5.00e+06_met_3.00e-02_to_3.00e-02.e%j	   # Name of stderr error file
#SBATCH -p small          # Queue (partition) name
#SBATCH -N 1             # Total # of nodes
#SBATCH -n 10             # Total # of mpi tasks
#SBATCH -t 48:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

export I_MPI_DEBUG=4

mkdir $WORK/cosmo_bh_grid/n_5.00e+06

# Launch MPI code...
mkdir $W

# h5copy commands

In [4]:
n_binaries = n_tot_binaries[5]
met_range = metallicities[:]

for met in met_range:
    print(f"$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_{n_binaries:.2e}/met_{met:.2e}_combined.h5 $WORK/cosmo_bh_grid/n_{n_binaries:.2e}/met_{met:.2e}/ -r ")

$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_1.00e-04_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_1.00e-04/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_1.88e-04_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_1.88e-04/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_3.55e-04_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_3.55e-04/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_6.69e-04_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_6.69e-04/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_1.26e-03_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_1.26e-03/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_2.38e-03_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_2.38e-03/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_4.48e-03_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_4.48e-03/ -r 
$WORK/h5copy.py -o $WORK/cosmo_bh_grid/n_5.00e+06/met_8.45e-03_combined.h5 $WORK/cosmo_bh_grid/n_5.00e+06/met_8.45e-03/ -r 
$WORK/h5